In [ ]:
# Question Answering with Memory using SPA

# Now we will build this model again, using the spa (semantic pointer 
# architecture) package built into Nengo 2.0.

# Press the play button to run the simulation.
# Graphs show the colour, shape and cue inputs. The last graph
# shows that the output is most similar to the semantic pointer which was 
# initially bound to the given cue. For example, when SQUARE is 
# provided as a cue, the output is most similar to BLUE.

import nengo
import nengo_spa as spa
from nengo_spa import Vocabulary
import numpy as np

d = 32  # the dimensionality of the vectors
rng = np.random.RandomState(7)
vocab = Vocabulary(dimensions=d, pointer_gen=rng, max_similarity=0.1)

#Adding semantic pointers to the vocabulary
vocab.populate("CIRCLE;BLUE;RED;SQUARE")
vocab.add('ZERO', [0]*d)

with spa.Network(label="Question Answering with Memory", vocabs=[vocab]) as model:

    #function for providing color input
    def color_input(t):
        if t < 0.25:
            return 'RED'
        elif t < 0.5:
            return 'BLUE'
        else:
            return 'ZERO'

    #function for providing shape input
    def shape_input(t):
        if t < 0.25:
            return 'CIRCLE'
        elif t < 0.5:
            return 'SQUARE'
        else:
            return 'ZERO'

    #function for providing the cue
    def cue_input(t):
        if t < 0.5:
            return 'ZERO'
        sequence = ['ZERO', 'CIRCLE', 'RED', 'ZERO', 'SQUARE', 'BLUE']
        idx = int(((t - 0.5) // (1. / len(sequence))) % len(sequence))
        return sequence[idx]
    
    #Inputs
    A = spa.Transcode(color_input, label="colour", output_vocab=d)
    B = spa.Transcode(shape_input, label="shape", output_vocab=d)
    C = spa.Transcode(cue_input, label="cue", output_vocab=d)
    
    #States
    D = spa.State(d, label="bound")
    E = spa.State(d, label="output")
    memory = spa.State(d, feedback=1, label="memory")
    
    #Actions
    A * B >> D
    D >> memory
    memory * ~C >> E
    